In [1]:
from huggingface_hub import list_repo_refs
out = list_repo_refs("allenai/OLMo-2-1124-7B")
branches = [b.name for b in out.branches]
# print(branches)

/home/user/repos/ModelWeave/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for branch in branches:
    print(branch)

stage2-ingredient3-step11000-tokens47B
stage2-ingredient3-step10000-tokens42B
stage2-ingredient3-step9000-tokens38B
stage2-ingredient3-step8000-tokens34B
stage2-ingredient3-step7000-tokens30B
stage2-ingredient3-step6000-tokens26B
stage2-ingredient3-step5000-tokens21B
stage2-ingredient3-step4000-tokens17B
stage2-ingredient3-step3000-tokens13B
stage2-ingredient3-step2000-tokens9B
stage2-ingredient3-step1000-tokens5B
stage2-ingredient2-step1000-tokens5B
stage2-ingredient2-step2000-tokens9B
stage2-ingredient2-step3000-tokens13B
stage2-ingredient2-step4000-tokens17B
stage2-ingredient2-step5000-tokens21B
stage2-ingredient2-step6000-tokens26B
stage2-ingredient2-step7000-tokens30B
stage2-ingredient2-step8000-tokens34B
stage2-ingredient2-step9000-tokens38B
stage2-ingredient2-step10000-tokens42B
stage2-ingredient2-step11000-tokens47B
stage2-ingredient1-step1000-tokens5B
stage2-ingredient1-step2000-tokens9B
stage2-ingredient1-step3000-tokens13B
stage2-ingredient1-step4000-tokens17B
stage2-ingredi

In [3]:
branches_stage1 = [branch for branch in branches if 'stage2' not in branch][1:]
branches_stage1_healed = [f'stage1-{branch}' if 'stage1' not in branch else branch for branch in branches_stage1]
for branch in branches_stage1_healed:
    print(branch)

stage1-step98000-tokens412B
stage1-step198000-tokens831B
stage1-step191000-tokens802B
stage1-step186000-tokens781B
stage1-step175000-tokens735B
stage1-step101000-tokens424B
stage1-step249000-tokens1045B
stage1-step250000-tokens1049B
stage1-step251000-tokens1053B
stage1-step252000-tokens1057B
stage1-step253000-tokens1062B
stage1-step254000-tokens1066B
stage1-step255000-tokens1070B
stage1-step256000-tokens1074B
stage1-step257000-tokens1078B
stage1-step258000-tokens1083B
stage1-step259000-tokens1087B
stage1-step260000-tokens1091B
stage1-step261000-tokens1095B
stage1-step262000-tokens1099B
stage1-step263000-tokens1104B
stage1-step264000-tokens1108B
stage1-step265000-tokens1112B
stage1-step266000-tokens1116B
stage1-step267000-tokens1120B
stage1-step268000-tokens1125B
stage1-step269000-tokens1129B
stage1-step270000-tokens1133B
stage1-step271000-tokens1137B
stage1-step272000-tokens1141B
stage1-step273000-tokens1146B
stage1-step274000-tokens1150B
stage1-step275000-tokens1154B
stage1-step276000

In [4]:
import re

def extract_step(branch_name):
    match = re.search(r'step(\d+)', branch_name)
    return int(match.group(1)) if match else float('inf')

branches_stage1_healed_sorted = sorted(branches_stage1_healed, key=extract_step)
for branch in branches_stage1_healed_sorted:
    print(branch)

stage1-step150-tokens1B
stage1-step600-tokens3B
stage1-step700-tokens3B
stage1-step850-tokens4B
stage1-step900-tokens4B
stage1-step1000-tokens5B
stage1-step2000-tokens9B
stage1-step2150-tokens10B
stage1-step3000-tokens13B
stage1-step4000-tokens17B
stage1-step5000-tokens21B
stage1-step6000-tokens26B
stage1-step7000-tokens30B
stage1-step8000-tokens34B
stage1-step9000-tokens38B
stage1-step10000-tokens42B
stage1-step11000-tokens47B
stage1-step12000-tokens51B
stage1-step13000-tokens55B
stage1-step14000-tokens59B
stage1-step15000-tokens63B
stage1-step16000-tokens68B
stage1-step17000-tokens72B
stage1-step18000-tokens76B
stage1-step19000-tokens80B
stage1-step20000-tokens84B
stage1-step21000-tokens89B
stage1-step22000-tokens93B
stage1-step23000-tokens97B
stage1-step24000-tokens101B
stage1-step25000-tokens105B
stage1-step26000-tokens110B
stage1-step27000-tokens114B
stage1-step28000-tokens118B
stage1-step29000-tokens122B
stage1-step30000-tokens126B
stage1-step31000-tokens131B
stage1-step32000-tok

hello


In [8]:
# Extract step numbers from branches_stage1_healed_sorted
steps = [extract_step(branch) for branch in branches_stage1_healed_sorted]

# Identify steps that don't follow the 1,000 interval schema
unexpected_steps = [step for step in steps if step % 1000 != 0]

# Identify missing steps in the sequence
expected_steps = set(range(0, max(steps) + 1000, 1000))
missing_steps = sorted(expected_steps - set(steps))

print("Unexpected steps:")
print(unexpected_steps)

print("\nMissing steps:")
print(missing_steps)

Unexpected steps:
[150, 600, 700, 850, 900, 2150, 928646, 928646]

Missing steps:
[0, 100000, 174000, 317000, 536000, 605000, 632000, 672000, 929000]
